### Notebook for initializing polymer systems using a DPD potential

In [32]:
import matplotlib
import numpy as np  
import freud
import gsd, gsd.hoomd 
import hoomd 
import time
import matplotlib_inline
import matplotlib.pyplot as plt
from gmso.core.topology import Topology
from gmso.external import from_mbuild, to_gsd_snapshot, to_hoomd_forcefield, to_hoomd_snapshot

from gmso.parameterization import apply

%matplotlib inline
matplotlib.style.use("ggplot")
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")

In [37]:
def initialize_snapshot_rand_walk(num_pol, num_mon, density=0.85, bond_length=1.0, buffer=0.1):
    '''
    Create a HOOMD snapshot of a cubic box with the number density given by input parameters.
    '''    
    N = num_pol * num_mon
    L = np.cbrt(N / density)  # Calculate box size based on density
    positions = np.zeros((N, 3))
    for i in range(num_pol):
        start = i * num_mon
        positions[start] = np.random.uniform(low=(-L/2),high=(L/2),size=3)
        for j in range(num_mon - 1):
            delta = np.random.uniform(low=(-bond_length/2),high=(bond_length/2),size=3)
            delta /= np.linalg.norm(delta)*bond_length
            positions[start+j+1] = positions[start+j] + delta
    positions = pbc(positions,[L,L,L])
    bonds = []
    for i in range(num_pol):
        start = i * num_mon
        for j in range(num_mon - 1):
            bonds.append([start + j, start + j + 1])
    bonds = np.array(bonds)
    frame = gsd.hoomd.Frame()
    frame.particles.types = ['A']
    frame.particles.N = N
    frame.particles.position = positions
    frame.bonds.N = len(bonds)
    frame.bonds.group = bonds
    frame.bonds.types = ['b']
    frame.configuration.box = [L, L, L, 0, 0, 0]
    return frame

def pbc(d,box):
    for i in range(3):
        a = d[:,i]
        a[a < -box[i]/2] += box[i]
        a[a >  box[i]/2] -= box[i]
    return d

def check_bond_length_equilibration(snap,num_mon,num_pol,max_bond_length=1.1,min_bond_length=1.0): #todo
    frame_ds = []
    for j in range(num_pol):
        idx = j*num_mon
        d1 = snap.particles.position[idx:idx+num_mon-1] - snap.particles.position[idx+1:idx+num_mon]
        bond_l = np.linalg.norm(pbc(d1,snap.configuration.box),axis=1)
        frame_ds.append(bond_l)
    max_frame_bond_l = np.max(np.array(frame_ds))
    min_frame_bond_l = np.min(np.array(frame_ds))
    print("max: ",max_frame_bond_l," min: ",min_frame_bond_l)
    if max_frame_bond_l <= max_bond_length and min_frame_bond_l >= min_bond_length:
        print("Bonds relaxed.")
        return True
    if max_frame_bond_l > max_bond_length or min_frame_bond_l < min_bond_length:
        return False

def run_one(A=1000,gamma=1000,k=1000,num_pol=100,num_mon=10,kT=1.0,r_cut = 1.15,bond_l=1.0,dt=0.001,density=0.8,particle_spacing = 1.1
):
    print(num_pol*num_mon)
    print(f"\nRunning with A={A}, gamma={gamma}, k={k}, "
          f"num_pol={num_pol}, num_mon={num_mon}")
    start_time = time.perf_counter()
    frame = random_walk(chain_L=num_mon,num_chains=num_pol,radius=0.01,bond_L=bond_l,density=density)
    build_stop = time.perf_counter()
    print("Total build time: ", build_stop-start_time)
    harmonic = hoomd.md.bond.Harmonic()
    harmonic.params["b"] = dict(r0=bond_l, k=k)
    integrator = hoomd.md.Integrator(dt=dt)
    integrator.forces.append(harmonic)
    simulation = hoomd.Simulation(device=hoomd.device.auto_select(), seed=np.random.randint(65535))# TODO seed
    simulation.operations.integrator = integrator 
    simulation.create_state_from_snapshot(frame)
    const_vol = hoomd.md.methods.ConstantVolume(filter=hoomd.filter.All())
    integrator.methods.append(const_vol)
    nlist = hoomd.md.nlist.Cell(buffer=0.4)
    simulation.operations.nlist = nlist
    DPD = hoomd.md.pair.DPD(nlist, default_r_cut=r_cut, kT=kT)
    DPD.params[('A', 'A')] = dict(A=A, gamma=gamma)
    integrator.forces.append(DPD)
    
    simulation.run(0)
    simulation.run(1000)
    snap=simulation.state.get_snapshot()
    
    while not check_bond_length_equilibration(snap,num_mon, num_pol,max_bond_length=particle_spacing): #TODO - work with snapshot instead of gsd?
        check_time = time.perf_counter()
        if (check_time-start_time) > 60:
            return num_pol*num_mon, 0
        simulation.run(1000)
        snap=simulation.state.get_snapshot()
    '''
    while not check_inter_particle_sep(snap):
        simulation.run(100)
        snap=simulation.state.get_snapshot()
    '''
    end_time = time.perf_counter()
    return num_pol*num_mon, end_time - start_time
    
N,s = run_one(A=1000,gamma=800,k=20000,num_pol=100,num_mon=50) #A=1000 seems fast, cranking up gamma to 800 speeds up, cranking up k more does too,
#no difference between k=10k and 20k
#OK, with these parameters, we're finishing in under 100 steps
print(f"Finished in time = {s:.2f}s")

5000

Running with A=1000, gamma=800, k=20000, num_pol=100, num_mon=50
Finished walk 0
Finished walk 1
Finished walk 2
Finished walk 3
Finished walk 4
Finished walk 5
Finished walk 6
Finished walk 7
Finished walk 8
Finished walk 9
Finished walk 10
Finished walk 11
Finished walk 12
Finished walk 13
Finished walk 14
Finished walk 15
Finished walk 16
Finished walk 17
Finished walk 18
Finished walk 19
Finished walk 20
Finished walk 21
Finished walk 22
Finished walk 23
Finished walk 24
Finished walk 25
Finished walk 26
Finished walk 27
Finished walk 28
Finished walk 29
Finished walk 30
Finished walk 31
Finished walk 32
Finished walk 33
Finished walk 34
Finished walk 35
Finished walk 36
Finished walk 37
Finished walk 38
Finished walk 39
Finished walk 40
Finished walk 41
Finished walk 42
Finished walk 43
Finished walk 44
Finished walk 45
Finished walk 46
Finished walk 47
Finished walk 48
Finished walk 49
Finished walk 50
Finished walk 51
Finished walk 52
Finished walk 53
Finished walk 54
Fini

*Warning*: Systems with many particle types perform poorly or result in shared memory errors on the GPU.
*Warning*: Systems with many bond types perform poorly or result in shared memory errors on the GPU.
*Warning*: Systems with many angle types perform poorly or result in shared memory errors on the GPU.
*Warning*: Systems with many dihedral types perform poorly or result in shared memory errors on the GPU.
*Warning*: Systems with many pair types perform poorly or result in shared memory errors on the GPU.


Total build time:  18.809740416996647


IncompleteSpecificationError: Error applying parameters for object of type <class 'hoomd.md.bond.Harmonic'>.

In [12]:
nlist

NameError: name 'nlist' is not defined

In [4]:
N,s = run_one(A=1000,gamma=800,k=20000,num_pol=100,num_mon=100)
print(f"Finished in time = {s:.2f}s")

10000

Running with A=1000, gamma=800, k=20000, num_pol=100, num_mon=100
Total build time:  0.076498584006913
max:  1.0327503255036956  min:  0.9704325493364084
DPD Simulation Finished.
Finished in time = 1.34s


In [52]:
N,s = run_one(A=1000,gamma=800,k=20000,num_pol=100,num_mon=1000)
print(f"Finished in time = {s:.2f}s")

100000

Running with A=1000, gamma=800, k=20000, num_pol=100, num_mon=1000
50.0
Total build time:  0.5120189579902217
max:  1.0339326  min:  0.967937
DPD Simulation Finished.
Finished in time = 22.36s


In [6]:
N,s = run_one(A=10000,gamma=800,k=20000,num_pol=200,num_mon=100)
print(f"Finished in time = {s:.2f}s")

20000

Running with A=10000, gamma=800, k=20000, num_pol=200, num_mon=100
Total build time:  0.10955829196609557
max:  1.1032426576806043  min:  0.8423931396572772
max:  1.0819145568037702  min:  0.9104981740053496
DPD Simulation Finished.
Finished in time = 5.31s


In [35]:
import matplotlib.pyplot as plt
import mbuild as mb
import logging
from mbuild import mBuildLogger
import numpy as np
import unyt as u

from mbuild.path import HardSphereRandomWalk, Path
from mbuild.utils.volumes import CuboidConstraint, SphereConstraint, CylinderConstraint
from mbuild.utils.density import rank_points_by_density, find_low_density_point

mbuild_logger = mBuildLogger()
mbuild_logger.library_logger.setLevel(logging.ERROR)

import random

def random_walk(chain_L,num_chains,radius,bond_L,density):
    N = chain_L * num_chains
    L = np.cbrt(N / density)
    cube = CuboidConstraint(center=(0,0,0), Lx=L, Ly=L, Lz=L)
    last_path = None
    
    for walk_num in range(num_chains):
        walk_passed = False
        while not walk_passed:
            try:
                if walk_num == 0:
                    initial_point = (0,0,0)
                else:
                  initial_point = find_low_density_point(
                        points=last_path.coordinates,
                        box_min=cube.mins,
                        box_max=cube.maxs,
                        edge_buffer=radius,
                        n_candidates=500,
                    )[np.random.randint(10)]
                
                path = HardSphereRandomWalk(
                    N=chain_L,
                    bead_name=f"_A{walk_num}",
                    radius=radius,
                    volume_constraint=cube,
                    bond_length=bond_L,
                    min_angle=np.pi/2,
                    max_angle=np.pi,
                    max_attempts=1e4,
                    start_from_path=last_path,
                    attach_paths=False,
                    seed=None,
                    initial_point=initial_point,
                    trial_batch_size=100
                )
                print(f"Finished walk {walk_num}")
                last_path = path
                walk_passed = True
            except Exception as e:
                print(e)
    
    polymer_system = last_path.to_compound()
    base_units = {
    "mass": u.g / u.mol, "length": u.nm, "energy": u.kJ / u.mol,
    }
    top = from_mbuild(polymer_system)
    top.identify_connections()
    snapshot, refs = to_hoomd_snapshot(top, base_units=base_units, auto_scale=False)

    return snapshot

In [ ]:
def check_for_overlap(positions, box, excluded_bond_depth, minimum_distance=0.10):
    """Check if snapshot contains overlapping particles.

    Parameters:
    -----------
    excluded_bond_depth : int, required
        The depth of bonded neighbors to exclude from overlap check.
        see Compound.direct_bonds()
    minimum_distance : float, default=0.10
        Distance (in nanometers) used as the threshold in
        determining if a pair of particles overlap.

    Notes:
    ------
    If `minimum_distance` is set larger than existing bond lengths,
    adjust the `excluded_bond_depth` parameter to excluded directly
    bonded neighbors from overlap checks.

    See Also:
    ---------
    mbuild.Compound.direct_bonds()

    Returns:
    --------
    overlapping_particles : list of tuples
        A list of particle pairs that were found within minimum_distance.
    """

    moved_positions, freud_box = self.to_freud()
    aq = freud.locality.AABBQuery(freud_box, moved_positions)
    aq_query = aq.query(
        query_points=moved_positions,
        query_args=dict(r_min=0.0, r_max=minimum_distance, exclude_ii=True),
    )
    nlist = aq_query.toNeighborList()
    # nlist contains each pair twice, get the set
    pairs_set = set([tuple(sorted((i, j))) for i, j in nlist])
    all_particles = [p for p in self.particles()]
    overlapping_particles = []
    for i, j in pairs_set:
        # Exclude bonded neighbors that are within min distance
        if excluded_bond_depth > 0:
            i_bonds = all_particles[i].direct_bonds(graph_depth=excluded_bond_depth)
            if all_particles[j] not in i_bonds:
                overlapping_particles.append((i, j))
        else:  # Don't exclude bonded neighbors
            overlapping_particles.append((i, j))
    return overlapping_particles

In [25]:
def to_freud(positions,box):
    """Convert a hoomd snapshot to a freud system (freud box and shifted coordinates)."""
    Lx,Ly,Lz = snap.configuration.box
    moved_positions = positions - np.array([Lx / 2, Ly / 2,Lz / 2])

    freud_box = freud.box.Box.from_matrix(box.vectors.T)

    freud_box.periodic = (True, True, True)
    return moved_positions, freud_box